In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchsummary import summary
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("D:\data\Student_performance_data _.csv")
df.sample(5)

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
130,1131,18,1,0,1,16.659883,9,0,1,0,0,0,0,2.104796,3.0
2271,3272,15,0,2,1,14.321261,25,1,3,0,0,0,0,1.302393,4.0
459,1460,17,0,0,3,0.876937,3,1,2,0,1,1,0,3.004233,1.0
846,1847,16,1,1,2,16.027041,23,1,3,1,1,0,1,1.767647,4.0
1814,2815,18,1,1,3,5.043640,29,1,2,1,0,0,0,0.210979,4.0


In [4]:
df.isnull().sum()

StudentID            0
Age                  0
Gender               0
Ethnicity            0
ParentalEducation    0
StudyTimeWeekly      0
Absences             0
Tutoring             0
ParentalSupport      0
Extracurricular      0
Sports               0
Music                0
Volunteering         0
GPA                  0
GradeClass           0
dtype: int64

In [6]:
#split dataset

#convert numpy arrays
X = df.drop(columns=['Ethnicity', 'StudentID']).values
y = df['Ethnicity'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print('X_train shape is : {}'.format(X_train.shape),'\n')
print('X_test  shape is : {}'.format(X_test.shape),'\n')
print('y_train shape is : {}'.format(y_train.shape),'\n')
print('y_test  shape is : {}'.format(y_test.shape),'\n')

X_train shape is : (1913, 13) 

X_test  shape is : (479, 13) 

y_train shape is : (1913,) 

y_test  shape is : (479,) 



In [9]:
type(X_train)

numpy.ndarray

In [10]:
data_loader = DataLoader(dataset = list(zip(X_train, y_train)),batch_size = 16, shuffle = True)
data, labels = next(iter(data_loader))

print("The shape of singular batch size of training : {}".format(data.size()))
print("The shape of singular batch size of testing  : {}".format(labels.size()))
     

The shape of singular batch size of training # torch.Size([16, 13])
The shape of singular batch size of testing  # torch.Size([16])


In [12]:
###ANN building

class StudentClassifier(nn.Module):
    def __init__(self):
        super(StudentClassifier, self).__init__()
        #create 1st hidden layer
        self.hidden_layer_1 = nn.Linear(in_features=13, out_features=128, bias=True)
        self.activation_1 = nn.ReLU()
        #create 2nd hidden layer
        self.hidden_layer_2 = nn.Linear(in_features=128, out_features=64, bias=True)
        self.activation_2 = nn.ReLU()
        #create output layer
        self.output_layer = nn.Linear(in_features=64, out_features=1, bias=True)
        self.activation_output = nn.Sigmoid()
        
    def forward(self, x):
        #activation(W^T + B); x = training dataset
        X = self.hidden_layer_1(x)
        X = self.activation_1(X)
        X = self.hidden_layer_2(X)
        X = self.activation_2(X)
        X = self.output_layer(X)
        X = self.activation_output(X)
        
        return X
    
    
model = StudentClassifier()
print(model, '\n\n\n')
print(model.parameters)

StudentClassifier(
  (hidden_layer_1): Linear(in_features=13, out_features=128, bias=True)
  (activation_1): ReLU()
  (hidden_layer_2): Linear(in_features=128, out_features=64, bias=True)
  (activation_2): ReLU()
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
  (activation_output): Sigmoid()
) 



<bound method Module.parameters of StudentClassifier(
  (hidden_layer_1): Linear(in_features=13, out_features=128, bias=True)
  (activation_1): ReLU()
  (hidden_layer_2): Linear(in_features=128, out_features=64, bias=True)
  (activation_2): ReLU()
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
  (activation_output): Sigmoid()
)>


In [17]:
print(model.parameters(),'\n\n\n')
summary(model, X_train.shape)

<generator object Module.parameters at 0x0000013ADBCE3220> 



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1            [-1, 1913, 128]           1,792
              ReLU-2            [-1, 1913, 128]               0
            Linear-3             [-1, 1913, 64]           8,256
              ReLU-4             [-1, 1913, 64]               0
            Linear-5              [-1, 1913, 1]              65
           Sigmoid-6              [-1, 1913, 1]               0
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.09
Forward/backward pass size (MB): 5.63
Params size (MB): 0.04
Estimated Total Size (MB): 5.77
----------------------------------------------------------------
